In [1]:
!curl -O https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (35) Recv failure: Connection was reset


In [2]:
with open("input.txt" , "r") as f:
    text = f.read()

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [ ]:
# charlevel tokenizer

# charactoer to integer mapping & integer to character mapping
stoi = { c:i for i , c in enumerate(chars) }
itos = { i:c for i , c in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s]  # encoder (string -> list of ints)
decode = lambda l: ''.join([itos[i] for i in l])  # decoder (list of ints -> string)

In [5]:
print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [ ]:
# encoding entire dataset
import torch

data = torch.tensor(encode(text) , dtype=torch.long)

print(data.shape , data.dtype)


torch.Size([1115393]) torch.int64


In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [12]:
block_size = 8

train_data[:block_size+1]  # first 9 characters in training set

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]  # first t+1 characters
    target = y[t]  # t+1th character

In [ ]:
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?

def get_batch(split):
    data = train_data if split == 'train' else val_data

    ix = torch.randint(len(data) - block_size , (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x,y

xb , yb = get_batch('train')

print(f'inputs: {xb.shape} , {xb}')
print(f'targets: {yb.shape} , {yb}')

inputs: torch.Size([4, 8]) , tensor([[ 0, 32, 46, 43, 52,  1, 58, 46],
        [63,  0, 13,  1, 54, 50, 39, 41],
        [ 0, 23, 43, 43, 54, 43, 56, 10],
        [46, 39, 58,  1, 57, 39, 44, 43]])
targets: torch.Size([4, 8]) , tensor([[32, 46, 43, 52,  1, 58, 46, 59],
        [ 0, 13,  1, 54, 50, 39, 41, 43],
        [23, 43, 43, 54, 43, 56, 10,  0],
        [39, 58,  1, 57, 39, 44, 43,  1]])


In [17]:
for b in range(batch_size):
    for t in range(block_size):

        context = xb[b, : t+1]
        target = yb[b, t]

        print(f'when input is {context} the target is {target}')

when input is tensor([0]) the target is 32
when input is tensor([ 0, 32]) the target is 46
when input is tensor([ 0, 32, 46]) the target is 43
when input is tensor([ 0, 32, 46, 43]) the target is 52
when input is tensor([ 0, 32, 46, 43, 52]) the target is 1
when input is tensor([ 0, 32, 46, 43, 52,  1]) the target is 58
when input is tensor([ 0, 32, 46, 43, 52,  1, 58]) the target is 46
when input is tensor([ 0, 32, 46, 43, 52,  1, 58, 46]) the target is 59
when input is tensor([63]) the target is 0
when input is tensor([63,  0]) the target is 13
when input is tensor([63,  0, 13]) the target is 1
when input is tensor([63,  0, 13,  1]) the target is 54
when input is tensor([63,  0, 13,  1, 54]) the target is 50
when input is tensor([63,  0, 13,  1, 54, 50]) the target is 39
when input is tensor([63,  0, 13,  1, 54, 50, 39]) the target is 41
when input is tensor([63,  0, 13,  1, 54, 50, 39, 41]) the target is 43
when input is tensor([0]) the target is 23
when input is tensor([ 0, 23]) th